In [20]:
import numpy as np
import pandas as pd
import tqdm 
import cProfile
import lightgbm as lgb
from sklearn.model_selection import GroupKFold, train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import logging
import time

data = pd.read_feather("massBV.feather")

In [19]:
data.head(20)

,ringnr,ID,mean_pheno,hatchisland,FID,PAT,MAT,SEX,PHENOTYPE,SNPa29779_A,...,SNPa447848_T,SNPa447849_A,SNPi31396_A,SNPi23364_C,SNPi32423_T,SNPi6538_G,SNPi43119_T,SNPa506292_T,SNPa506289_T,SNPa506288_G
0,8118424,0.451363,81.470000,28,1,8981850,8939392,0,-9,0.0,...,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,2.0
1,8118425,-1.114459,76.205000,28,1,8981850,8939392,0,-9,0.0,...,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
2,8118426,-1.299215,79.576667,28,1,0,8L48694,0,-9,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,8118429,0.683269,80.850000,28,1,8800436,8939615,0,-9,0.0,...,1.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,8118430,-1.063425,79.000000,28,1,8981850,8939392,0,-9,0.0,...,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
5,8118432,1.286515,80.620000,26,1,8887335,8L30782,0,-9,1.0,...,0.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0
6,8118433,-1.580081,79.575000,26,1,8887335,8L30782,0,-9,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
7,8118435,-0.980158,77.640000,26,1,0,0,0,-9,2.0,...,0.0,2.0,0.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0
8,8168602,-0.857946,77.360000,38,1,8732255,8732254,0,-9,0.0,...,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
9,8168603,0.925904,82.360000,38,1,8732255,8732254,0,-9,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0


In [2]:
def prep_data_before_train(data: pd.DataFrame, phenotype: str) -> tuple:
    """
    prepare data for training, returns target vector and covariate-matrix and ringnrs for grouping
    :param data: all data from dataloader script
    :param phenotype: the phenotype to be predicted
    :return: X, Y, ringnrs, X contain covariates and Y contains the phenotype
    """
    ringnrs = data.ringnr
    mean_pheno = data.mean_pheno
    X = data.drop(
        columns=[
            "ID",
            "mass",
            "tarsus",
            "ringnr",
            "mean_pheno",
            "IID",
            "FID",
            "MAT",
            "PAT",
            "SEX",
            "PHENOTYPE",
        ],
        errors="ignore",
    )

    try:
        Y = data.loc[:, phenotype]
    except KeyError:
        try:
            Y = data.ID
        except AttributeError:
            Y = data.mass
    del data
    try:
        X.hatchyear = X.hatchyear.astype("int")
        X.island_current = X.island_current.astype("int")
    except AttributeError:
        pass
    Y = (Y - np.mean(Y))/np.std(Y)
    X = X.fillna(0)
    X = X.T
    X = X.astype("int")
    X = X.T 
    return X, Y, ringnrs, mean_pheno

In [10]:
def splitter(X, ringnrs):
    # Split dataset based on ringnrs (i.e, dont want a individual to be present in both test and train set)
    kf = GroupKFold(n_splits=5)
    kfsplits = kf.split(X, groups=ringnrs)
    fold_names = [i for i in range(5)]
    for i, (train, test) in enumerate(kfsplits):
        yield train, test, fold_names[i]

In [3]:
def simple_splitter(X, y, proportion_of_snps : float = 1, test_size : float = 0.2): 
    #The proportion of snps to be used in the model. To have the option to use a smaller number of snps for testing and getting results faster
    np.random.seed(42)
    sample_columns = np.random.choice(X.shape[1], int(X.shape[1]*proportion_of_snps), replace=False)
    X = X.iloc[:, sample_columns]

    sample_rows = np.random.choice(X.shape[0], int(X.shape[0]*(1-test_size)), replace=False)
    X_train = X.iloc[sample_rows]
    y_train = y.iloc[sample_rows]
    X_test = X.drop(sample_rows)
    y_test = y.drop(sample_rows)


    return X_train, X_test, y_train, y_test, sample_rows
    


In [4]:
X, y, ringnrs, mean_pheno = prep_data_before_train(data, "mass")
X.drop(columns = ["hatchisland"], inplace = True)

In [11]:
X_train, X_test, y_train, y_test, sample_rows = simple_splitter(X, y, proportion_of_snps=0.1, test_size=0.2)
mean_pheno_test = mean_pheno.drop(sample_rows) #mean_pheno_test is the mean phenotype of the test set, to be measured against the predictions with pearson correlation

In [19]:
lgbm_space = {
    'n_estimators': np.arange(300, 600, 900),#[ 150, 200, 300, ],
    'learning_rate': np.arange(0.01, 0.3, 0.05),# [0.03, 0.05, 0.07, ],
    'max_depth': np.arange(3, 13),# [5, 7, 10, 15, 20,],
    'num_leaves': np.arange(20, 3000, 100),#[ 31, 50, 70, 100, 150, 200, ],
    #'min_gain_to_split': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1],
    #'min_child_weight': np.arange(0.02, 0.1, 0.01),#[  0.1, 0.2, ],
    #'min_child_samples': [ 30, 40, 50, 60, 80, 100, 120, 150, 170, 200, 300], 
    'reg_lambda': np.arange(1,15),
    'reg_alpha':  np.arange(1,15),
    'bagging_fraction': np.arange(0.2, 0.9, 0.1),#[0.3, 0.5,  0.8],  
    'baggings_freq': np.arange(1, 10, 1),#[ 3, 5, 7, 9, 11, 13, 15, 17, 19, 21],    
    'feature_fraction': np.arange(0.2, 0.9, 0.1)#[0.3, 0.5, 0.8],  
}

In [12]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define objective function for Optuna
def objective(trial):
    
    logger.info(f"Starting trial {trial.number}")

    # Define hyperparameter search space
    n_estimators = trial.suggest_int('n_estimators', 300, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    max_depth = trial.suggest_int('max_depth', 3, 13)
    num_leaves = trial.suggest_int('num_leaves', 20, 3000, step = 20)
    reg_lambda = trial.suggest_int('reg_lambda', 0, 100, step = 5)
    reg_alpha = trial.suggest_int('reg_alpha', 0 , 100, step = 5)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9, step = 0.1)
    bagging_freq = trial.suggest_int('bagging_freq', 1, 10, step = 1)
    feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9, step = 0.1)

    
    #parameters
    params = {
        'objective': 'regression',
        'metric': 'mse',    
        #'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'reg_lambda': reg_lambda,
        'reg_alpha': reg_alpha,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'feature_fraction': feature_fraction,
        'verbose': -1
    }
    
    # Cross-validation setup
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = [] # Store the mean squared error for each fold

    start_time = time.time()
    # Evaluate with cross-validation
    for train_idx, val_idx in kfold.split(X_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        # Train model
        dtrain = lgb.Dataset(X_train_fold, label=y_train_fold)
        dval = lgb.Dataset(X_val_fold, label=y_val_fold, reference=dtrain)
        
        gbm = lgb.train(params, dtrain, num_boost_round=n_estimators, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=20)])
        
        # Predict on validation set
        y_pred = gbm.predict(X_val_fold)
        mse = mean_squared_error(y_val_fold, y_pred)
        mse_scores.append(mse)

        mean_mse = np.mean(mse_scores)
        end_time = time.time()
        logger.info(f"Finished trial {trial.number} in {end_time - start_time:.2f} seconds")
        return mean_mse




if __name__ == '__main__':

    # Create Optuna study
    study = optuna.create_study(direction='minimize')

    n_jobs = 4
    
    study.optimize(objective, n_trials=10, n_jobs=n_jobs)

    # Best hyperparameters
    print('Best hyperparameters:', study.best_params)

[I 2024-09-12 08:40:22,058] A new study created in memory with name: no-name-057c49f5-a05c-4c3c-ad81-9a6011c6248b
INFO:__main__:Starting trial 0
INFO:__main__:Starting trial 1
INFO:__main__:Starting trial 2
INFO:__main__:Starting trial 3


Training until validation scores don't improve for 20 rounds
Training until validation scores don't improve for 20 rounds
Training until validation scores don't improve for 20 rounds
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[26]	valid_0's l2: 0.930644


INFO:__main__:Finished trial 3 in 11.49 seconds
[I 2024-09-12 08:40:33,733] Trial 3 finished with value: 0.9306442219195172 and parameters: {'n_estimators': 696, 'learning_rate': 0.16172110694705055, 'max_depth': 7, 'num_leaves': 2240, 'reg_lambda': 65, 'reg_alpha': 40, 'bagging_fraction': 0.8, 'bagging_freq': 10, 'feature_fraction': 0.9}. Best is trial 3 with value: 0.9306442219195172.
INFO:__main__:Starting trial 4


Early stopping, best iteration is:
[55]	valid_0's l2: 0.907236


INFO:__main__:Finished trial 0 in 16.63 seconds
[I 2024-09-12 08:40:38,862] Trial 0 finished with value: 0.9072361723012389 and parameters: {'n_estimators': 703, 'learning_rate': 0.20497929861378913, 'max_depth': 4, 'num_leaves': 1840, 'reg_lambda': 30, 'reg_alpha': 45, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 8, 'feature_fraction': 0.4}. Best is trial 0 with value: 0.9072361723012389.
INFO:__main__:Starting trial 5


...

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[19]	valid_0's l2: 0.932187


INFO:__main__:Finished trial 8 in 9.26 seconds
[I 2024-09-12 08:41:02,694] Trial 8 finished with value: 0.9321866002080017 and parameters: {'n_estimators': 591, 'learning_rate': 0.19778543649602615, 'max_depth': 8, 'num_leaves': 2660, 'reg_lambda': 35, 'reg_alpha': 20, 'bagging_fraction': 0.2, 'bagging_freq': 3, 'feature_fraction': 0.7}. Best is trial 5 with value: 0.8916446573487868.


Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[28]	valid_0's l2: 0.95802


INFO:__main__:Finished trial 9 in 10.14 seconds
[I 2024-09-12 08:41:07,557] Trial 9 finished with value: 0.9579288303155713 and parameters: {'n_estimators': 743, 'learning_rate': 0.29287839443495634, 'max_depth': 9, 'num_leaves': 720, 'reg_lambda': 45, 'reg_alpha': 85, 'bagging_fraction': 0.9, 'bagging_freq': 10, 'feature_fraction': 0.2}. Best is trial 5 with value: 0.8916446573487868.


Best hyperparameters: {'n_estimators': 493, 'learning_rate': 0.19900495462488169, 'max_depth': 13, 'num_leaves': 2440, 'reg_lambda': 80, 'reg_alpha': 35, 'bagging_fraction': 0.9, 'bagging_freq': 5, 'feature_fraction': 0.4}


In [8]:
plot_slice = optuna.visualization.plot_slice(study)
plot_slice.show()

In [13]:
best_params = study.best_params

n_estimators = best_params['n_estimators']
best_params.pop('n_estimators')
best_params['objective'] = 'regression'
best_params['metric'] = 'mse'
best_params['verbose'] = -1

print(f"Best params: {best_params}")

# Train model with best hyperparameters
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test, reference=dtrain)

gbm = lgb.train(best_params, dtrain, num_boost_round=n_estimators, valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=20)])

# Predict on test set
y_pred = gbm.predict(X_test)

# Calculate the pearson correlation between the predicted values and the maen phenotype of the test set
correlation = np.corrcoef(y_pred, mean_pheno_test)[0,1]



Best params: {'learning_rate': 0.19900495462488169, 'max_depth': 13, 'num_leaves': 2440, 'reg_lambda': 80, 'reg_alpha': 35, 'bagging_fraction': 0.9, 'bagging_freq': 5, 'feature_fraction': 0.4, 'objective': 'regression', 'metric': 'mse', 'verbose': -1}
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[77]	valid_0's l2: 0.927379


In [14]:
np.corrcoef(y_pred, mean_pheno_test)

array([[1.        , 0.23637552],
       [0.23637552, 1.        ]])

In [ ]:
core